## Analysis using Word2Vec and ANN

In [181]:
import numpy as np
import pandas as pd

In [97]:
# !pip install --upgrade pandas

In [98]:
df = pd.read_csv("https://raw.githubusercontent.com/sahaniaditya/Wikipedia_Toxic_Comment_Classification/main/final_dataset.csv")

In [99]:
df.head()

,Unnamed: 0,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,num_words,num_sent,num_characters,transformed_text
0,0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,49,3,264,explan edit made usernam hardcor metallica fan...
1,1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,27,4,112,match background colour seemingli stuck thank ...
2,2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,48,3,233,hey man realli tri edit war guy constantli rem...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,128,4,622,ca make real suggest improv wonder section sta...
4,4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,18,2,67,sir hero chanc rememb page


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Unnamed: 0        159571 non-null  int64 
 1   comment_text      159571 non-null  object
 2   toxic             159571 non-null  int64 
 3   severe_toxic      159571 non-null  int64 
 4   obscene           159571 non-null  int64 
 5   threat            159571 non-null  int64 
 6   insult            159571 non-null  int64 
 7   identity_hate     159571 non-null  int64 
 8   num_words         159571 non-null  int64 
 9   num_sent          159571 non-null  int64 
 10  num_characters    159571 non-null  int64 
 11  transformed_text  159441 non-null  object
dtypes: int64(10), object(2)
memory usage: 14.6+ MB


In [182]:
df = df.drop(columns=["Unnamed: 0"])

KeyError: "['Unnamed: 0'] not found in axis"

In [102]:
df.head() ## dropped the unecessary columns

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,num_words,num_sent,num_characters,transformed_text
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,49,3,264,explan edit made usernam hardcor metallica fan...
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,27,4,112,match background colour seemingli stuck thank ...
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,48,3,233,hey man realli tri edit war guy constantli rem...
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,128,4,622,ca make real suggest improv wonder section sta...
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,18,2,67,sir hero chanc rememb page


In [103]:
## Using Word2Vec to get the embeddings

In [105]:
df["transformed_text"] = df["transformed_text"].fillna("fuck you").astype(str)

In [106]:
sentences_lst = df["transformed_text"].to_list()

In [172]:
print(len(sentences_lst))

159571


In [108]:
import nltk

In [109]:
nltk.download("punkt")

In [110]:
from nltk.tokenize import word_tokenize

In [131]:
sentences_lst[852]

'fuck you'

In [179]:
processed_data = []
for sentence in sentences_lst:
  # print(word_tokenize(sentence))
  # print(i)
  # i = i + 1
  processed_data.append(word_tokenize(sentence))

In [180]:
print(len(processed_data))

159571


In [183]:
from gensim.models import Word2Vec

In [184]:
for idx, sentence in enumerate(processed_data):
    if type(sentence) != list:
        processed_data[idx] = [sentence]

In [185]:
model = Word2Vec(sentences=processed_data, vector_size=1000, window=5, min_count=1, workers=4)

In [186]:
model.save("word2vec.model")

In [208]:
model = Word2Vec.load("word2vec.model")
word_vector = model.wv['fuck']
print("Vector for 'fuck':", word_vector.shape)

RuntimeError: DataLoader worker (pid 18458) is killed by signal: Killed. 

In [188]:
def get_features(sentence, model):
  vectors = [model.wv[word] for word in sentence if word in model.wv]
  if len(vectors) == 0:
    return np.zeros(1000)
  return np.mean(vectors, axis=0)

In [189]:
feature_vectors = [get_features(sentence, model) for sentence in processed_data]

In [190]:
features = np.array(feature_vectors)

In [191]:
features.shape

(159571, 1000)

In [192]:
## implementing ANN using PyTorch

In [193]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [194]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
from torchvision import datasets, transforms

In [196]:
## creating custom dataset and dataloader
class MyDataset(Dataset):
  def __init__(self, data, label, transform = None):
    self.data = data
    self.labels = label
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    sample = self.data[idx]
    label = self.labels[idx]

    if self.transform:
            sample = self.transform(sample)

    return sample, label

In [197]:
print(len(df["toxic"].to_list()))

159571


In [198]:
data = features
labels = df["toxic"].to_list()
transform = None
dataset = MyDataset(data, labels, transform)

In [199]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)

In [200]:
class ANN(nn.Module):
  def __init__(self, dropout, num_classes):
    super(ANN, self).__init__()
    self.fc1 = nn.Linear(1000, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 32)
    self.fc4 = nn.Linear(32, num_classes)
    self.relu = nn.ReLU()
    self.classes = num_classes
    self.dropout = nn.Dropout(p=dropout)

  def forward(self, x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.dropout(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.dropout(x)
    x = self.fc3(x)
    x = self.relu(x)
    x = self.fc4(x)

    return x

In [201]:
model = ANN(0.2, 2)

In [202]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [203]:
num_epochs = 5

In [204]:
## training the model
for epoch in range(num_epochs):
  model.train()
  total_loss = 0.0


  for batch_idx, (data, targets) in enumerate(dataloader):
    # zero the gradients

    optimizer.zero_grad()
    outputs = model(data)
    loss = criterion(outputs, targets)

    loss.backward()
    optimizer.step()

    total_loss += loss.item() / 32

  print("Epoch : ", epoch," ", total_loss)

Epoch :  0   27.250941813872487
Epoch :  1   25.296237997928984
Epoch :  2   24.721683512514574
Epoch :  3   24.770269791190003
Epoch :  4   24.256260983936954


In [205]:
from sklearn.metrics import f1_score

In [211]:
def evaluate_model(model, test_loader):
  model.eval()
  correct = 0
  total = 0
  total_f1 = 0.0

  with torch.no_grad(): # disable gradient computation
     for data, targets in test_loader:
      outputs = model(data)
      _, predicted = torch.max(outputs.data, 1)
      total += 32
      correct += (predicted == targets).sum().item()
      # print(predicted)
      f1 = f1_score(targets, predicted, average="binary", zero_division=0)
      total_f1 += (f1 / 32)



  accuracy = 100 * correct / total
  print(f'Test Accuracy: {accuracy:.2f}%')
  print(f'F1 Score: {total_f1:.2f}%')

In [212]:
evaluate_model(model, dataloader)

Test Accuracy: 94.76%
F1 Score: 91.27%
